# Imports:

In [13]:
!pip3 install mediapipe-silicon opencv-python

ERROR: Could not find a version that satisfies the requirement mediapipe-silicon (from versions: none)
ERROR: No matching distribution found for mediapipe-silicon

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Code: 

### Imports

In [14]:
import cv2
import mediapipe as mp 
import numpy as np 
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

### Calculate Angle

In [15]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

### Dectections

In [25]:
# setup camera
cam = cv2.VideoCapture(0)

# squat counter variables
counter = 0 
stage = None


#setup mediapipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cam.isOpened():
        ret, frame = cam.read()
        
        # BRG -> RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        results = pose.process(image)

        # Get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
    
        # RGB -> BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # get landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # show angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(knee, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )             
        except:
            pass
        
        # draw detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('p'):
            break

    cam.release()
    cv2.destroyAllWindows()

Testing:

In [24]:
landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility
landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]

hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

hip, knee, ankle

calculate_angle(hip, knee, ankle)


178.88150714028825

### Clean Up:

In [17]:
cam.release()
cv2.destroyAllWindows()